In [1]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
import tracks_interactions.graph.family_graph as fg


In [2]:
os.environ['NAPARI_PERFMON'] = '0'

In [3]:
from tifffile import imread
labels = imread(r'D:\kasia\tracking\E6_exp\sample_data\track_labels.tif')

In [4]:
# get images
ch1 = imread(r'D:\kasia\tracking\E6_exp\sample_data\ch1.tif')
ch2 = imread(r'D:\kasia\tracking\E6_exp\sample_data\ch2.tif')

In [5]:
# checkout labels as editable array
labels = np.array(labels).astype('int32')

In [6]:
# display channels and labels
viewer = napari.Viewer()
viewer.add_image(ch1, name='ch1', colormap = 'green',blending='additive')
viewer.add_image(ch2, name='ch2', colormap = 'red',blending='additive')
labels_layer = viewer.add_labels(labels,name='Labels')

In [7]:
# get access to the database
new_db_path =r'D:\kasia\tracking\E6_exp\sample_data\sample_db_2tables.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

### Family graph display

In [8]:
from tracks_interactions.graph.family_graph import FamilyGraphWidget
plot_widget = FamilyGraphWidget(viewer,session)
viewer.window.add_dock_widget(plot_widget,area='bottom')

### Navigation widget

In [9]:
from tracks_interactions.widget.track_navigation_widget import TrackNavigationWidget
track_navigation_widget = TrackNavigationWidget(viewer,session)
viewer.window.add_dock_widget(track_navigation_widget,area='right',name='Track Navigation')

### Modification widget

In [10]:
from tracks_interactions.widget.track_modification_widget import TrackModificationWidget
track_modification_widget = TrackModificationWidget(viewer,session)
viewer.window.add_dock_widget(track_modification_widget,area='right',name='Track Modifications')

In [11]:
#session.query(CellDB).filter(CellDB.track_id == 9).all()

In [12]:
#session.query(TrackDB).filter(TrackDB.track_id == 91).all()

### Signal graph

In [13]:
from tracks_interactions.graph.signal_graph import SignalGraph
signal_list = ['ch0_nuc','ch0_cyto']
color_list = ['yellow','green']
graph_widget = SignalGraph(viewer,session,signal_list,color_list)
viewer.window.add_dock_widget(graph_widget,area='right',name='Cell Signals')

In [14]:
# # add a button controling adding graphs
# signal_list = ['area','ch0_nuc','ch0_cyto','ch1_nuc','ch1_cyto']

# from tracks_interactions.widget.signal_graph_widget import AddListGraphWidget
# add_graph_widget = AddListGraphWidget(viewer,session,signal_list)
# viewer.window.add_dock_widget(add_graph_widget,area='right',name='Add Signal Graphs')

### Cell modification

In [15]:
from tracks_interactions.widget.cell_modification_widget import CellModificationWidget
cell_modification_widget = CellModificationWidget(viewer,session,ch_list=[ch1,ch2])
viewer.window.add_dock_widget(cell_modification_widget,area='right',name='Save Cell')

### Tests

In [16]:
# session.close()

In [17]:
# cell = session.query(CellDB).filter(CellDB.t==0).filter(CellDB.track_id==34).all()
# cell = cell[0]
# print(cell.row,cell.col)
# print(cell.bbox_0,cell.bbox_1,cell.bbox_2,cell.bbox_3)
# print(cell.tags)